In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import sys
import os
!pip install ekphrasis
!pip install tweet-preprocessor
!pip install textblob
!python -m textblob.download_corpora

dir_of_interest = '/content/drive/MyDrive/Rumor Detection/utils'
# modules = {}

sys.path.insert(0 , dir_of_interest)
# for module in os.listdir(dir_of_interest):
#     if '.py' in module and '.pyc' not in module:
#         current = module.replace('.py', '')
#         modules[current] = __import__(current)


from Feature_extraction import *
from collect_tweets import *
# from clean_data import *

import sys
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
!pip install ekphrasis
!pip install tweet-preprocessor
import preprocessor as p
import re
from textblob import TextBlob
import seaborn as sns
from scipy.stats import pointbiserialr

!pip install textblob
!python -m textblob.download_corpora

%matplotlib inline

     |████████████████████████████████| 81kB 4.4MB/s 
     |████████████████████████████████| 184kB 12.7MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=4a23088110dfa163b9cc303824efd771337d22343193c94bca384580e99f5646
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=46255c5dfbf93c68eb080845259a039a3899e8d0d2db03faf034814bdd2821fd
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built ekphrasis ftfy
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]  

In [15]:
tc = TweetCollector()
tc.get_data_from_id('/content/drive/MyDrive/Rumor Detection/Dataset/live test', [1410339585083772930, 1408837952889933827])

downloading...: 2it [00:00,  5.81it/s]

2 number of tweets collected


[]

In [16]:
fc = FeatureExtractor('/content/drive/MyDrive/Rumor Detection/Dataset/live test')
tweet , user = fc.get_data(None)

tweet['created_at'] = tweet['created_at'].apply(lambda x: str(x))
user['created_at'] = user['created_at'].apply(lambda x: str(x))
user[['followers_count', 'statuses_count','friends_count',
      'favourites_count', 'listed_count',]] = user[['followers_count', 'statuses_count','friends_count',
      'favourites_count', 'listed_count']].astype(np.int64)

tweet[['retweet_count','favorite_count','n_symbols', 'n_user_mentions',
       'n_hashtags']] = tweet[['retweet_count','favorite_count','n_symbols', 'n_user_mentions',
       'n_hashtags']].astype(np.int64)

Reading...: 100%|██████████| 2/2 [00:00<00:00, 45.08it/s]


In [21]:
tweet

,tweetId,text,source,created_at,is_reply,retweet_count,favorite_count,is_quote_status,entities_symbols,n_symbols,entities_user_mentions,n_user_mentions,entities_hashtags,n_hashtags,entities_url,n_url,label
0,1410339585083772930,Ex-US defence secretary Donald Rumsfeld dead a...,SocialFlow,2021-06-30 20:48:54,None,86,253,False,[],0,[],0,[],0,[https://bbc.in/3whEA4N],1,None
1,1408837952889933827,Matt Hancock resigns as health secretary after...,SocialFlow,2021-06-26 17:21:57,None,4802,21701,False,[],0,[],0,[],0,[https://bbc.in/3dka2bU],1,None


In [22]:
user

,userId,name,screen_name,tweetId,verfifed,followers_count,statuses_count,friends_count,favourites_count,listed_count,location,created_at,profile_image_url,profile_background_image_url,default_profile_image,default_profile,profile_use_background_image,url
0,742143,BBC News (World),BBCWorld,1410339585083772930,True,32188761,329553,18,13,124575,"London, UK",2007-02-01 07:44:29,http://pbs.twimg.com/profile_images/1150717770...,http://abs.twimg.com/images/themes/theme1/bg.png,False,False,True,https://t.co/7NEgoMwJy3
1,5402612,BBC Breaking News,BBCBreaking,1408837952889933827,True,47882735,37316,3,0,141703,"London, UK",2007-04-22 14:42:37,http://pbs.twimg.com/profile_images/1150716997...,http://abs.twimg.com/images/themes/theme1/bg.png,False,False,True,http://t.co/vBzl7LOaso


In [25]:
 def meta_data_preprocessing(tweet, user):
  import numpy as np
  user['created_at'] = user['created_at'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  tweet['created_at'] = tweet['created_at'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  current_datetime = datetime.now()
  user['account_age'] = user['created_at'].apply(lambda x : (current_datetime - x).days)
  user['screen_name_len'] = user['screen_name'].apply(len)
  user['verfifed'] = user['verfifed'].astype(np.int64)
  user['default_profile'] = user['default_profile'].astype(np.int64)
  user['default_profile_image'] = user['default_profile_image'].astype(np.int64)
  user['profile_use_background_image'] = user['profile_use_background_image'].astype(np.int64)
  user['profile_image_url'] = user['profile_image_url'].apply(lambda x : x is not np.nan )
  user['profile_image_url'] = user['profile_image_url'].astype(np.int64)
  user['profile_background_image_url'] = user['profile_background_image_url'].apply(lambda x : x is not np.nan )
  user['profile_background_image_url'] = user['profile_background_image_url'].astype(np.int64)
  user['has_location'] = user['location'].apply(lambda x : x is not np.nan)
  user['has_location'] = user['has_location'].astype(np.int64)
  user['has_url'] = user['url'].apply(lambda x : x is not np.nan)
  user['has_url'] = user['has_url'].astype(np.int64)
  user[['followers_count', 'statuses_count','friends_count',
      'favourites_count', 'listed_count', 'account_age']] = user[['followers_count', 'statuses_count','friends_count',
      'favourites_count', 'listed_count', 'account_age']].apply(lambda x  : np.log(x+1))

  # cleaning text data 

  for i,v in enumerate(tweet['text']):
    tweet.loc[i,'text'] = p.clean(v)

  def preprocess_data(data):
    #Removes Numbers
    data = data.astype(str).str.replace('\d+', '')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()
  
    def lemmatize_text(text):
      return [(lemmatizer.lemmatize(w)) for w \
                          in w_tokenizer.tokenize((text))]

    def remove_punctuation(words):
      new_words = []
      for word in words:
        new_word = re.sub(r'[^\w\s]', '', (word))
        if new_word != '':
          new_words.append(new_word)
      return new_words

    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

  pre_tweets = preprocess_data(tweet['text'])
  tweet['text'] = pre_tweets
  stop_words = set(stopwords.words('english'))
  tweet['text'] = tweet['text'].apply(lambda x: [item for item in \
                                      x if item not in stop_words])
  
  print()
  def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    return analysis.sentiment.polarity

  tweet['polarity'] =  tweet['text'].apply(lambda x : analyze_sentiment(' '.join(x)))
  tweet['is_reply'] = tweet['is_reply'].isnull()
  tweet['is_reply'] = tweet['is_reply'].apply(lambda x : not x)
  tweet['is_reply'] = tweet['is_reply'].astype(np.int64)
  tweet['is_quote_status'] = tweet['is_quote_status'].astype(np.int64)
  tweet['text_len'] = tweet['text'].apply(len)
  tweet['post_age'] = tweet['created_at'].apply(lambda x : (current_datetime-x).days)
  tweet[['retweet_count','favorite_count','n_symbols', 'n_user_mentions',
       'n_hashtags']] = tweet[['retweet_count','favorite_count','n_symbols', 'n_user_mentions',
       'n_hashtags']].apply(lambda x : np.log(x+1))
  merge = tweet.merge(user, on='tweetId', suffixes=['_tweet', '_user'])
  merge['posted_in'] = merge['created_at_tweet'] - merge['created_at_user']
  merge['posted_in'] =  merge['posted_in'].apply(lambda x : x.days)
  df = merge[['is_reply','verfifed', 'followers_count',
       'retweet_count', 'favorite_count', 'is_quote_status', 'n_symbols', 
       'n_user_mentions', 'n_hashtags','n_url', 'polarity', 'text_len', 'post_age',
       'statuses_count', 'friends_count', 'favourites_count', 'listed_count',
       'profile_image_url', 'profile_background_image_url', 'default_profile_image',
       'default_profile', 'profile_use_background_image', 'account_age',
       'screen_name_len', 'has_location', 'has_url', 'posted_in', 'label']]
  # df.loc[:, 'label'] = df['label'].replace({'non-rumor':False, 'true':True, 'false':True, 'unverified':True})
  selected_features = ['profile_use_background_image','profile_background_image_url',
                     'default_profile_image','default_profile','verfifed', 'text_len',
                     'n_url', 'post_age', 'statuses_count', 'listed_count', 'n_symbols',
                     'posted_in', 'n_user_mentions', 'polarity', 'favourites_count', 'favorite_count',
                     'screen_name_len', 'n_hashtags']
  sel_num_features = ['text_len','n_url', 'post_age', 'statuses_count', 'listed_count', 'n_symbols',
                     'posted_in', 'n_user_mentions', 'polarity', 'favourites_count', 'favorite_count',
                     'screen_name_len', 'n_hashtags']

  sel_cat_features = ['profile_use_background_image','profile_background_image_url',
                      'default_profile_image','default_profile','verfifed']
  X = df[selected_features]
  y = df['label']
  print(X)
  from pickle import load
  sc_x = load(open('/content/drive/MyDrive/Rumor Detection/Scaler/scaler.pkl', 'rb'))
  X[sel_num_features] = sc_x.transform(X[sel_num_features])
  return X, y

In [26]:
X_user_tweet, y_user_tweet = meta_data_preprocessing(tweet.copy(), user.copy())
X_user_tweet


Empty DataFrame
Columns: [profile_use_background_image, profile_background_image_url, default_profile_image, default_profile, verfifed, text_len, n_url, post_age, statuses_count, listed_count, n_symbols, posted_in, n_user_mentions, polarity, favourites_count, favorite_count, screen_name_len, n_hashtags]
Index: []


ValueError: ignored